In [1]:
import pandas as pd
import os
import gzip
import nibabel as nib
import re
import numpy as np

/var/folders/kb/xjgrybvs26qc31wnr88cf93w0000gn/T/ipykernel_1214/1644224066.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Create table

In [ ]:
dicc = {'ecc':[],'area':[],'R2':[],'rfsize':[],'ang':[], 'subj':[], 'hem':[]}

pattern = r'^(lh|rh)\.fit1_(\w+)\.mgz$'

path_area =['/Users/mariedelvalle/Documents/HCP/atlasmgz/lh.Wang2015.mgz', '/Users/mariedelvalle/Documents/HCP/atlasmgz/rh.Wang2015.mgz']
area = []

for p in path_area:
    data = nib.load(p)
    data = np.squeeze(data.get_fdata())

    area.extend(data)

path = "/Users/mariedelvalle/Documents/HCP/prfresultsmgz"

subjects =  os.listdir(path)

for i,sub in enumerate(subjects):

    if sub.isdigit() and len(sub) == 6:

        print(f"{sub} - {i}/{len(subjects)}")

        sub_files = sorted(os.listdir(f"{path}/{sub}"))

        for sub_file in sub_files:
            match = re.match(pattern, sub_file)

            if match:
                hem = match.group(1)
                prop = match.group(2)

                data = nib.load(f'{path}/{sub}/{sub_file}')
                data = np.squeeze(data.get_fdata())

                if prop in dicc.keys():
                    dicc[prop].extend(data)
    
        dicc['subj'].extend([sub]* (2* data.shape[0]))
        dicc['hem'].extend([0]*data.shape[0])
        dicc['hem'].extend([1]*data.shape[0])
        dicc['area'].extend(area)

df = pd.DataFrame(dicc)
df = df.dropna()

# Guardar el DataFrame como un archivo CSV
df.to_csv('HCP_prfresults_no_nan.csv', index=False)



# Change Wang atlas

In [14]:
path = '/Users/mariedelvalle/Documents/HCP/atlasmgz'
files = os.listdir(path)

pattern = r'^(lh|rh)\.Wang2015\.mgz$'

atlas_lh = []
atlas_rh = []


for file in files:
    
    match = re.match(pattern, file)
    if match:
        hem = match.group(1)
        data = nib.load(f'{path}/{file}')
        print(data.get_data_dtype())
        data = np.squeeze(data.get_fdata())

        if hem == 'lh':
            atlas_lh.extend(np.round(data).astype(int))
        else:
            atlas_rh.extend(np.round(data).astype(int))


# Change rois

mapping = {0:0, 1: 1, 2: 1, 3: 2, 4: 2, 5: 3, 6: 3}

for i in range(4,23):
    mapping[i + 3] = i


new_atlas_lh = np.vectorize(lambda x: mapping[x])(atlas_lh)
new_atlas_rh = np.vectorize(lambda x: mapping[x])(atlas_rh)

image_lh = nib.Nifti1Image(new_atlas_lh.reshape(163842, 1, 1).astype(float), np.eye(4))
image_rh = nib.Nifti1Image(new_atlas_rh.reshape(163842, 1, 1).astype(float), np.eye(4))


# Guardar la imagen como archivo .mgz
nib.save(image_lh, 'lh.Wang2015_Modif.mgz')
nib.save(image_rh, 'rh.Wang2015_Modif.mgz')
        



>f4
>f4


/var/folders/kb/xjgrybvs26qc31wnr88cf93w0000gn/T/ipykernel_1214/902383926.py:22: RuntimeWarning: invalid value encountered in cast
  atlas_rh.extend(np.round(data).astype(int))
/var/folders/kb/xjgrybvs26qc31wnr88cf93w0000gn/T/ipykernel_1214/902383926.py:20: RuntimeWarning: invalid value encountered in cast
  atlas_lh.extend(np.round(data).astype(int))


In [20]:
data = nib.load('lh.Wang2015_Modif.mgz')
print(data.get_fdata().shape)      
data = np.squeeze(data.get_fdata())

print(data.shape)
print(np.unique(data))


(163842, 1, 1)
(163842,)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22.]
